<div id="toc"></div>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Goal

The goal of this work is to create methods in `bicorr.py` that will enable the following capabilities in analyzing `bicorr_hist_master`.

Issues:  
1) Change the time bins to be more coarse  
2) Normalizing the counts in each bin so that the magnitude in each bin is approximately the same regardless of the time bin width

Start by importing relevant packages and some data.

In [ ]:
import numpy as np
import scipy.io as sio
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.colors
import inspect

In [ ]:
# Load the bicorr.py functions I have already developed
sys.path.append('../scripts')

import bicorr as bicorr

In [ ]:
%load_ext autoreload
%autoreload 2

Import data from the following folder:

In [ ]:
os.listdir('../analysis/Cf072115_to_Cf072215b/')

In [ ]:
sparse_bhm, dt_bin_edges, note = bicorr.load_sparse_bhm(r'../analysis/Cf072115_to_Cf072215b/')

In [ ]:
det_df = bicorr.load_det_df('../meas_info/det_df_pairs_angles.csv')

I'm going to work with the positive `bhm` only and not perform a background subtraction.

In [ ]:
bhm = bicorr.revive_sparse_bhm(sparse_bhm, det_df, dt_bin_edges)

Save the `bicorr_hist_plot` for coarsening later.

In [ ]:
bhp = bicorr.build_bhp(bhm, dt_bin_edges, print_flag=True)[0]

# Issue 1: Make time binning more coarse on `bhm`

In some instances, there are not enough counts in each histogram to see smooth features, and therefore it is beneficial to make the time bins more coarse (change the time bin width from 0.25 ns to 1 ns, for example).

Here I will write a function that performs that task based on a specified factor $C$ by which to coarsen the time bin width. If a histogram with time bin width 0.25 ns is coarsend by a factor $C=4$, then the resulting histogram will have time bin widths of $0.25*4$ ns $= 1$ ns.

## Generate coarse matrix

What is the shape and bin width of the original matrix?

In [ ]:
shape = bhm.shape
print('Dimensions of bicorr_hist_master: ', bhm.shape)

dt_bin_width = dt_bin_edges[1]-dt_bin_edges[0]
print('Width of time bin in (ns): ', dt_bin_width)

What will be the shape of the final matrix?

In [ ]:
C = 4

# Preallocate coarse matrix
bhm_coarse = np.zeros((shape[0],shape[1],int(shape[2]/C),int(shape[3]/C)))

# Calculate new dt_bin_edges
dt_bin_edges_coarse = dt_bin_edges[0::C]
dt_bin_width_coarse = dt_bin_edges_coarse[1]-dt_bin_edges_coarse[0]

print('Dimensions of bicorr_hist_master_coarse: ', bhm_coarse.shape)
print('Width of coarse time bin in (ns): ', dt_bin_width_coarse)

To fill the histogram, I need to sum over a range of values in `bicorr_hist_master`. If the coarsening factor is $C=4$, then the corresponding bin indices are:

Index in `bicorr_hist_master_coarse` --> Indices in `bicorr_hist_master`...  

0 --> (0-3)  
1 --> (4-7)  
2 --> (8-11 )  
3 --> (12-15)  
4 --> (16-19)  

But in python, to select data at indices $0-3$, for instance, you must use the index range `[0:4]`, so the indices that you call will be even simpler.

For an index $i$ in `bicorr_hist_master_coarse`, the corresponding starting and ending bin indices in `bicorr_hist_master` given factor $C$ are:

Starting index: `C*i`  
Final index:    `C*(i+1)`  

Loop through all of the bins in `bicorr_hist_master_coarse` and fill them by summing the corresponding bins in `bicorr_hist_master`:

In [ ]:
for bin1 in np.arange(0,bhm_coarse.shape[2]):
    for bin2 in np.arange(0,bhm_coarse.shape[3]):
        bhm_coarse[:,:,bin1,bin2] = np.sum(bhm[:,:,C*bin1:C*(bin1+1),C*bin2:C*(bin2+1)],axis=(2,3))

## Plot original and coarse matrices

Plot the two distributions to compare them. Start with all events across all detectors.

In [ ]:
plt.pcolormesh(dt_bin_edges, dt_bin_edges, np.sum(bhm,axis=(0,1)), norm = matplotlib.colors.LogNorm())
plt.xlabel('$\Delta t_1$ (ns)')
plt.ylabel('$\Delta t_2$ (ns)')
plt.title('Bicorr_hist_master with 0.25 ns bin width')
plt.colorbar()
plt.show()

In [ ]:
plt.pcolormesh(dt_bin_edges_coarse, dt_bin_edges_coarse, np.sum(bhm_coarse,axis=(0,1)), norm = matplotlib.colors.LogNorm())
plt.xlabel('$\Delta t_1$ (ns)')
plt.ylabel('$\Delta t_2$ (ns)')
plt.title('Bicorr_hist_master_coarse with 1.0 ns bin width')
plt.colorbar()
plt.show()

## Functionalize it

In [ ]:
print(inspect.getsource(bicorr.coarsen_bhm))

Try it out- make an even coarser matrix with 4 ns bins.

In [ ]:
bhm_4ns, dt_bin_edges_4ns = bicorr.coarsen_bhm(bhm,dt_bin_edges,16)

In [ ]:
plt.pcolormesh(dt_bin_edges_4ns, dt_bin_edges_4ns, np.sum(bhm_4ns,axis=(0,1)), norm = matplotlib.colors.LogNorm())
plt.xlabel('$\Delta t_1$ (ns)')
plt.ylabel('$\Delta t_2$ (ns)')
plt.title('bhm_coarse with 4.0 ns bin width')
plt.colorbar()
plt.show()

# Issue 2: Make time binning more coarse on `bhp`

In some cases I will already have `bhp` constructed, and I want to coarsen that instead of starting from `bhm`. Follow the same steps. 

## Generate `bhp_coarse`

In [ ]:
shape = bhp.shape
print('Dimensions of bicorr_hist_plot: ', bhp.shape)

dt_bin_width = dt_bin_edges[1]-dt_bin_edges[0]
print('Width of time bin in (ns): ', dt_bin_width)

In [ ]:
C = 4

# Preallocate coarse matrix
bhp_coarse = np.zeros((int(shape[0]/C),int(shape[1]/C)))

# Calculate new dt_bin_edges
dt_bin_edges_coarse = dt_bin_edges[0::C]
dt_bin_width_coarse = dt_bin_edges_coarse[1]-dt_bin_edges_coarse[0]

print('Dimensions of bicorr_hist_plot_coarse: ', bhp_coarse.shape)
print('Width of coarse time bin in (ns): ', dt_bin_width_coarse)

In [ ]:
for bin1 in np.arange(0,bhp_coarse.shape[0]):
    for bin2 in np.arange(0,bhp_coarse.shape[1]):
        bhp_coarse[bin1,bin2] = np.sum(bhp[C*bin1:C*(bin1+1),C*bin2:C*(bin2+1)],axis=(0,1))

## Plot original and coarse `bhp`

In [ ]:
bicorr.bicorr_plot(bhp, dt_bin_edges, show_flag = True)

In [ ]:
bicorr.bicorr_plot(bhp_coarse, dt_bin_edges_coarse, show_flag = True)

In [ ]:
print(inspect.getsource(bicorr.coarsen_bhp))

## Plot it directly

In [ ]:
bicorr.bicorr_plot(bhp,dt_bin_edges,show_flag = True, title='Original')

In [ ]:
bicorr.bicorr_plot(bicorr.coarsen_bhp(bhp,dt_bin_edges,4)[0],
                   bicorr.coarsen_bhp(bhp,dt_bin_edges,4)[1],
                   show_flag = True, title='Coarsen with C = 4')

In [ ]:
bicorr.bicorr_plot(bicorr.coarsen_bhp(bhp,dt_bin_edges,16)[0],
                   bicorr.coarsen_bhp(bhp,dt_bin_edges,16)[1],
                   show_flag = True, title='Coarsen with C = 16')

# Issue 3: Normalize the counts

When looking at these distributions in terms of number of counts, there are several issues that make it difficult to make comparisons between plots. These include the following points:

* With finer time bin width, there are fewer counts per bin. You will notice above that the scale on the color bar starts at over $10^4$ for 0.25 ns bins, goes above $10^5$ for 1 ns bins, and then above $10^6$ for 4 ns bins. This makes sense because the number of counts in each bin is increasing roughly with $C^2$ as the counts over a range of bins are combined. 
* So far in this analysis we have only generated plots for all detector pairs, but later we will generate this histogram for subsets of detector pairs based on the angle between detectors. Thus, a subset with fewer detector pairs would have fewer counts in each bin than a subset with more detector pairs.
* Also, some of the measurements were taken for longer measurement times, so that there are more fission events in the measurement, and therefore we would expect more counts.
* Finally, we will eventually compare these bicorrelation plots between measurement and simulation, so we need units that can be fairly compared between the two. 

To deal with this, we will normalize the number of counts in each bin of the histogram to the following things:

* Number of fission events
* Number of detector pairs
* Area of pixel in time-squared units

## Units 

These values will change for each measurement, but the information related to the number of fissions that I have is:

* fission rate in [fissions/s]
* measurement length in [s]

I can multiply these together to get the total number of fissions during the time of the measurement. Working with values from the `Cf072115-Cf072215b` measurement...

In [ ]:
fission_rate = 2.9498e+05 # fissions/s
meas_length = 7440 # sec
num_fissions = fission_rate * meas_length
print(num_fissions)

For the number of detector pairs, I can take the length of the array `pair_is` that I pass into `bicorr.build_bicorr_hist_plot`. `pair_is` contains an array of the indices of the selected detector pairs in `bicorr_hist_master`. 

In [ ]:
pair_is = det_df.index.values

Lastly, we need to calculate the size of the bin width in two-dimensional space. For example, the data in a bin of size 0.25 ns x 0.25 ns should be inflated by a factor of 16 (or divided by 1/16) in order to be compared to the data in a bin of size 1 ns x 1 ns.

In [ ]:
# Determine the time-binning
time_bin_width = dt_bin_edges[1]-dt_bin_edges[0]
time_norm_factor = np.power(time_bin_width,2) # Units of ns^2
print(time_bin_width, time_norm_factor)

## Calculate the normalization factor

Ultimately, the normalization factor is the product of these four values.

In [ ]:
norm_factor = num_fissions * len(pair_is) * time_norm_factor
print(norm_factor)

Try it now for the coarser bin widths and see if the two norm_factors differ by a factor of 16.

In [ ]:
norm_factor_coarse = num_fissions * len(pair_is) * time_norm_factor_coarse
print(norm_factor_coarse)

In [ ]:
print(norm_factor_coarse / norm_factor)

In [ ]:
C**2

Does this add up? The number of counts will be divided by the normalization factor. The coarser measurements should be divided by a greater value, therefore `norm_factor_coarse > norm_factor`. So it checks out.

## Compare normal and coarse plots after normalization

The colorbar scale on the two plots should be the same after normalization for any time bin width.

To normalize, divide the `bicorr_hist_plot` (summed `bicorr_hist_matrix`) matrix by `norm_factor`.

In [ ]:
plt.pcolormesh(dt_bin_edges, dt_bin_edges, np.divide(np.sum(bhm,axis=(0,1)),norm_factor), norm = matplotlib.colors.LogNorm())
plt.xlabel('$\Delta t_1$ (ns)')
plt.ylabel('$\Delta t_2$ (ns)')
plt.title('bhm with 0.25 ns bin width, normalized')
plt.axes().set_aspect('equal')
plt.colorbar()
plt.show()

In [ ]:
plt.pcolormesh(dt_bin_edges_coarse, dt_bin_edges_coarse, np.divide(np.sum(bhm_coarse,axis=(0,1)),norm_factor_coarse), norm = matplotlib.colors.LogNorm())
plt.xlabel('$\Delta t_1$ (ns)')
plt.ylabel('$\Delta t_2$ (ns)')
plt.title('Bicorr_hist_master_coarse with 1.0 ns bin width, normalized')
plt.axes().set_aspect('equal')
plt.colorbar()
plt.show()

In [ ]:
time_bin_width_4ns = dt_bin_edges_4ns[1]-dt_bin_edges_4ns[0]
time_norm_factor_4ns = np.power(time_bin_width_4ns,2) # Units of ns^2
norm_factor_4ns = num_fissions * len(pair_is) * time_norm_factor_4ns

plt.pcolormesh(dt_bin_edges_4ns, dt_bin_edges_4ns, np.divide(np.sum(bhm_4ns,axis=(0,1)),norm_factor_4ns), norm = matplotlib.colors.LogNorm())
plt.xlabel('$\Delta t_1$ (ns)')
plt.ylabel('$\Delta t_2$ (ns)')
plt.title('Bicorr_hist_master_coarse with 4.0 ns bin width, normalized')
plt.axes().set_aspect('equal')
plt.colorbar()
plt.show()

The 4 ns range is lower than the other two ranges. Why? This plot will have more data in each bin, but will be less susceptible to small fluctuations. For example, if there is a localized peak in the finer mesh plot, the average of those counts is lower, and the coarsened distribution will be representative of the average. 

## Confirm accuracy of normalization with variable `pair_is` and `type_is`

Change the number of detector pairs in each plot and verify that the normalization accounts for that. Work with the data binned to 1 ns time bins.

In [ ]:
time_bin_width_coarse = dt_bin_edges_coarse[1]-dt_bin_edges_coarse[0]
time_norm_factor_coarse = np.power(time_bin_width_coarse,2) # Units of ns^2
print(time_bin_width_coarse, time_norm_factor_coarse)

Start with all events and detector pairs for reference